# The Battle of Neighborhoods (Week 2)

In [141]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from IPython.display import display_html
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [142]:
!pip install lxml

#  Introduction/Business Problem

### A fresh graduate who has been staying in Toronto city since his high school. The place Toronto has been a very convenient one for him. The neighborhood he had around him was a blessings for him. Due to some unavoidable reasons he has to shift to Manhattan city in Newyork. This is both a matter of excitement and a bit worries for him. He has been habituated with lifestyle of Toronto and has known the city very well. Suddenly now he needs to face a change. On top it, he has to leave his part time job in his neighborhood in a coffee shop which basically somehow fulfills his regular needs. So, he needs to know the neighborhood of Manhattan about how similar it is to Toronto or how much different it is. Based on that he needs to take proper steps after he settles up.
## Problem Specification:               
### A comparison on the neighbourhood of Manhattan and Toronto will be presented here based on popular venues around them which would help find possible job opportunities for the newcomers, ammeture entrapreuers and investors.

#  Data Section for Toronto city

### The necessary data for Toronto city has been collected from wikipedia which is given with respect to postal codes with their neighborhoods. The data has been cleaned and all the places around Toronto has been extracted from this information with proper location data as well.

In [144]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list=pd.read_html(url,header=0)
df=df_list[0]
df.head()
df.shape

(180, 3)

### Deleting any row containing Not assigned in "Borough"

In [145]:
df_1=df.drop(df[df['Borough']=='Not assigned'].index).reset_index(drop=True)

### Making Not assigned "Neighborhood" to be the same as the respective "Borough"

In [146]:
rows_NA=df_1.loc[df_1.Neighborhood == 'Not assigned'].index
df_1['Neighborhood'][rows_NA]=df_1['Borough'][rows_NA]

### Inserting multiple "Neighborhood"s in a single row separated by commas having same "Postal Code"

In [147]:
df_2=df_1.groupby("Postal Code").agg(lambda x:','.join(set(x))).reset_index()
df_2.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Getting the Lattitude and Longitudes of places around Toronto:

In [148]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")

In [149]:
df_2['Latitude']=geo_data['Latitude'].values
df_2['Longitude']=geo_data['Longitude'].values

In [150]:
Toronto_data = df_2[df_2['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#  Data Section for Manhattan City

### The necessary data for Manhattan city has been downloaded from the open datasource  https://cocl.us/new_york_dataset has been cleaned and all the places around Manhattam has been extracted from this information with proper location data as well.

In [151]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [152]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [153]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [154]:
neighborhoods_data = newyork_data['features']

In [155]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [156]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [157]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_m = location.latitude
longitude_m = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_m, longitude_m))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


# Exploring the neighborhoods in Toronto and Manhattan

### Here, various neighbourhood spots in the Toronto and Manhattan city has been displayed on the map for visual aid. This also helps to understand the proximity of the considered placed considered.

### Creating a map of Toronto city with neighbourhoods:

In [158]:

latitude = Toronto_data.Latitude.mean()
longitude= Toronto_data.Longitude.mean()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Creating a map of neighbourhood of Manhattan:

In [159]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude_m, longitude_m], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### It seems that the neighbourhood considered is more scattered in Toronto compared to Manhattan which might be due to geographical difference specially in shape and surrounded river around Manhattan

 ### The Foursquare API has been utilized to explore the neighborhoods place in both cities to collect all popular venues around them which is eventually used to segment the two regions.

In [160]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: DXXF1AFZHL3JOEYD5KWUPQUX0TVOVIMUTA3TDDSNL34WDWVP
CLIENT_SECRET:OLINTY25VUFUOGCJW2NNCYKBKOOHOMLBWS4F552QSI1PN4IN


###  A function has been built to get the venues name and their location indicating latitude and longitude for all the neighborhoods in both the cities

In [161]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### After Running the above function on each neighborhood, two new dataframes called toronto_venues and manhattan_venues have been created which holds all the venue names, location and their catagory as well

In [163]:
print("\033[1m Venues nearby Toronto: \033[0m")
toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )
print("\033[1m Venues nearby Manhattan: \033[0m")
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


 Venues nearby Toronto: 
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Villag

## Analyzing Each Neighborhood

### Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category in toronto

In [164]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.052632
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.014925,0.0,0.0,0.014925,0.0,0.0,0.014925


### Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category in manhattan

In [165]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.000000,0.015152,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.015152,0.0,0.0,0.000000
1,Carnegie Hill,0.0,0.0,0.0,0.000000,0.011236,0.0,0.0,0.0,0.011236,...,0.0,0.022472,0.0,0.0,0.0,0.011236,0.033708,0.0,0.0,0.033708
2,Central Harlem,0.0,0.0,0.0,0.041667,0.041667,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
3,Chelsea,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.010000,0.010000,0.0,0.0,0.000000
4,Chinatown,0.0,0.0,0.0,0.000000,0.030000,0.0,0.0,0.0,0.000000,...,0.0,0.030000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.010000


### Printing each neighborhood along with the top 5 most common venues in toronto

In [166]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.03
3          Café  0.03
4   Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1         Breakfast Spot  0.08
2  Performing Arts Venue  0.08
3            Coffee Shop  0.08
4   Gym / Fitness Center  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.05
2               Brewery  0.05
3                   Spa  0.05
4            Smoke Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2          Coffee Shop  0.06
3  Rental Car Location  0.06
4   Airport Food Court  0.06


----Central Bay Street----
            

### Printing each neighborhood along with the top 5 most common venues in manhattan

In [167]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.12
1          Hotel  0.06
2    Coffee Shop  0.06
3  Boat or Ferry  0.05
4  Memorial Site  0.05


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.04
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4                 Gym  0.03


----Central Harlem----
                  venue  freq
0    Chinese Restaurant  0.06
1  Gym / Fitness Center  0.04
2    African Restaurant  0.04
3   American Restaurant  0.04
4           Pizza Place  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.08
1          Art Gallery  0.07
2       Ice Cream Shop  0.04
3  American Restaurant  0.04
4    French Restaurant  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.08
1              Bakery  0.07
2        Dessert Shop  0.04
3     Bubble Tea Shop  0.03
4                 Bar  0.03


----Civic Center----
                  venue  freq
0     

### All these datas will be put into a pandas dataframe and the venues will be sorted on their frequency or popularity values 

### A function to sort the venues in descending order:

In [168]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### A new dataframe and display the top 10 venues for each neighborhood in toronto

In [169]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_t = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_t['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted_t.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_t.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Café,Beer Bar,Japanese Restaurant,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Breakfast Spot,Coffee Shop,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boutique,Harbor / Marina,Plane,Coffee Shop,Boat or Ferry,Sculpture Garden,Rental Car Location,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Salad Place,Dessert Shop,Middle Eastern Restaurant,Thai Restaurant,Department Store


### Creating a new dataframe and display the top 10 venues for each neighborhood in manhattan

In [170]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_m = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_m['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted_m.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_m.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Memorial Site,Gym,Boat or Ferry,Food Court,Shopping Mall,Gourmet Shop,Mexican Restaurant
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Italian Restaurant,Yoga Studio,Bookstore,Wine Shop,Gym / Fitness Center,Gym,French Restaurant
2,Central Harlem,Chinese Restaurant,Seafood Restaurant,Pizza Place,Bar,French Restaurant,Gym / Fitness Center,American Restaurant,African Restaurant,Deli / Bodega,Bookstore
3,Chelsea,Coffee Shop,Art Gallery,Ice Cream Shop,American Restaurant,French Restaurant,Bakery,Café,Cocktail Bar,Pizza Place,Market
4,Chinatown,Chinese Restaurant,Bakery,Dessert Shop,Vietnamese Restaurant,Cocktail Bar,Bar,Spa,Ice Cream Shop,Bubble Tea Shop,Hotpot Restaurant


# Clustering Neighbourhoods

### Afterwards clustering method k-means has been used to cluster the neighbourhood into 5 clusters in both of the cities and they have been displayed in different colors in the map

In [171]:
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### For our ease of operation a new dataframe has been created that includes the cluster as well as the top 10 venues for each neighborhood

In [172]:
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
toronto_merged = Toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted_t.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Frozen Yogurt Shop,Ice Cream Shop,Furniture / Home Store,Liquor Store,Spa,Japanese Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Pub,Light Rail Station,Liquor Store,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Intersection
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Final resulting clusters visualization - Toronto Neighborhoods

In [173]:
# create map
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Similar analysis has been done for the manhattan city

In [174]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 1, 0, 1, 1, 2, 0, 0], dtype=int32)

In [175]:
# add clustering labels
neighborhoods_venues_sorted_m.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted_m.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Sandwich Place,Coffee Shop,Gym,Pharmacy,Department Store,Diner,Discount Store,Donut Shop,Kids Store,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Dessert Shop,Vietnamese Restaurant,Cocktail Bar,Bar,Spa,Ice Cream Shop,Bubble Tea Shop,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Bakery,Pizza Place,Grocery Store,Mobile Phone Shop,Donut Shop,Mexican Restaurant,Sandwich Place,Bank,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Lounge,Café,Restaurant,Park,Frozen Yogurt Shop,Bakery,Spanish Restaurant,Chinese Restaurant,American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Deli / Bodega,Coffee Shop,Café,Mexican Restaurant,Bakery,Sushi Restaurant,Cocktail Bar,Sandwich Place,Yoga Studio


In [176]:
# create map
map_clusters = folium.Map(location=[latitude_m, longitude_m], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [177]:
ct1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [178]:
cm1=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

# Cluster 2

In [179]:
ct2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [180]:
cm2=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

# Cluster 3

In [181]:
ct3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [182]:
cm3=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

# Cluster 4

In [183]:
ct4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [184]:
cm4=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

# Cluster 5

In [185]:
ct5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [186]:
cm5=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

## Results and Comparison:

### Popluar Venues in various clustors of Toronto

In [187]:
ct1['1st Most Common Venue'].value_counts()

Coffee Shop           9
Park                  4
Café                  4
Pizza Place           2
Clothing Store        2
Home Service          1
Hotel                 1
Breakfast Spot        1
Greek Restaurant      1
Light Rail Station    1
Mexican Restaurant    1
Bakery                1
Tennis Court          1
Trail                 1
Grocery Store         1
Sandwich Place        1
Pub                   1
Bar                   1
Name: 1st Most Common Venue, dtype: int64

In [188]:
ct2['1st Most Common Venue'].value_counts()

Coffee Shop    1
Name: 1st Most Common Venue, dtype: int64

In [189]:
ct3['1st Most Common Venue'].value_counts()

Café           1
Coffee Shop    1
Name: 1st Most Common Venue, dtype: int64

In [190]:
ct4['1st Most Common Venue'].value_counts()

Coffee Shop    1
Name: 1st Most Common Venue, dtype: int64

In [191]:
ct5['1st Most Common Venue'].value_counts()

Airport Service    1
Name: 1st Most Common Venue, dtype: int64

### For the Toronto city, as we can see the cluster 1 is very much well equipped with all catagories of shops that a person would need in daily basis. So, for a regular dewelling a newcomer person should chose any houses in cluster 1. Nevertheless, as these places are popular it seems,this might be populous compared to the other cluster zones. So, the persons who prefers more calm surroundings should choose other clusters. Also for job seekers, it would be obvious that cluster 1 would offer more openings compared to the other zones due to its popularity. Also for the entrepreneurs or one planning to start any small business, they should choose something which is not so common and popular in these regions such as gift shop/Italian restaurents.

### Popluar Venues in various clustors of Manhattan

In [192]:
cm1['1st Most Common Venue'].value_counts()

Coffee Shop           3
Bar                   2
Pizza Place           1
Chinese Restaurant    1
Italian Restaurant    1
Name: 1st Most Common Venue, dtype: int64

In [193]:
cm2['1st Most Common Venue'].value_counts()

Coffee Shop             3
Korean Restaurant       1
Sandwich Place          1
Café                    1
Park                    1
Theater                 1
Hotel                   1
Gym / Fitness Center    1
Name: 1st Most Common Venue, dtype: int64

In [194]:
cm3['1st Most Common Venue'].value_counts()

Mexican Restaurant    2
Café                  2
Chinese Restaurant    2
Bookstore             1
Sandwich Place        1
Park                  1
Deli / Bodega         1
Pizza Place           1
Name: 1st Most Common Venue, dtype: int64

In [195]:
cm4['1st Most Common Venue'].value_counts()

Italian Restaurant      8
Bakery                  1
Gym / Fitness Center    1
Name: 1st Most Common Venue, dtype: int64

In [196]:
cm5['1st Most Common Venue'].value_counts()

Park    1
Name: 1st Most Common Venue, dtype: int64

### Similarly for manhattan city cluster 4 would be a better choice for living provided it has somehow most of regular thing although not compared to cluster 1 in toronto. Specially, No popular or 1st most common choice health care service and groccery stores doesn't seem to there. There should be something in 2nd, 3rd or even further most common choice venues.
### One noticeable thing there is plenty of italian restaurents in manhattan in several clusters compared to Toronto city.

## Conclusion

### As we can see, with proper comparison between various parameters in our case for example: the 1st most common venues can provide several thoughts in chosing a proper place for living, proper idea for starting a new business and for understanding the people's liking in the area. Data analysis can make a big contribution such decisons effectively provided that we have sufficient amount of relevant, useful and up to date datas. 